# Dynamic WSCC 9-bus System 

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_RX_Dyn/WSCC-09_RX'
filename = 'WSCC-09'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy
import numpy as np


In [ ]:
sim_name = 'WSCC-9bus_dyn'

## Simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system = reader.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.TransientStability)

logger = dpsimpy.Logger(sim_name)

for i in range(1, 10):
    logger.log_attribute('v' + str(i), 'v', system.node('BUS' + str(i)))

for i in range(1, 4):
    logger.log_attribute('wr_' + str(i), 'w_r', system.component('GEN' + str(i)))

sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(system)
sim.set_time_step(0.0001)
sim.set_final_time(2.0)
sim.set_domain(dpsimpy.Domain.DP)
sim.do_steady_state_init(True)

sim.add_logger(logger)
sim.run()


## Read simulation results

In [ ]:
path = 'logs/WSCC-9bus_dyn/'
logName = 'WSCC-9bus_dyn_InitLeftVector'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim_init = rt.read_timeseries_dpsim(logFilename)

### Phasors at first time step

In [ ]:
phasors_init = ts.phasors(ts_dpsim_init)

for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[0]))

### Phasors at last time step

In [ ]:
for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

### Difference

In [ ]:
for node, phasor in phasors_init.items():
    print(node + ': ' + str(phasor['abs'].values[-1] - phasor['abs'].values[0]) + '<' + str(phasor['phase'].values[-1] - phasor['phase'].values[0]))

## Simulation

In [ ]:
path = 'logs/WSCC-9bus_dyn/'
logName = 'WSCC-9bus_dyn'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim = rt.read_timeseries_dpsim(logFilename)
phasors = ts.phasors(ts_dpsim)

### Phasors at last time step in per unit

In [ ]:
nominal_voltages = {'v1': 16500, 'v2': 18000, 'v3': 13800, 
                         'v4': 230000, 'v5': 230000, 'v6': 230000, 
                         'v7': 230000, 'v8': 230000, 'v9': 230000} 
for node, nom_voltage in nominal_voltages.items():
    print(node + ': ' + str(phasors[node]['abs'].values[0] / nom_voltage) + '<' + str(phasors[node]['phase'].values[0]))

### Plot node phase


In [ ]:
pt.plot_timeseries(1, phasors['v9']['phase'])
#plt.xlim(0,0.2)

### Plot node voltages

In [ ]:
for node, phasor in phasors.items():
    print(node + ': ' + str(phasor['abs'].values[-1]) + '<' + str(phasor['phase'].values[-1]))

In [ ]:
pt.plot_timeseries(1, phasors['v1']['abs'])
pt.plot_timeseries(1, phasors['v2']['abs'])
pt.plot_timeseries(1, phasors['v3']['abs'])

pt.plot_timeseries(2, phasors['v4']['abs'])
pt.plot_timeseries(2, phasors['v5']['abs'])
pt.plot_timeseries(2, phasors['v6']['abs'])
pt.plot_timeseries(2, phasors['v7']['abs'])
pt.plot_timeseries(2, phasors['v8']['abs'])
pt.plot_timeseries(2, phasors['v9']['abs'])
plt.ylim([200000,240000])
plt.xlim([0,0.1])

In [ ]:
for node, phasor in phasors.items():
    abs_diff = phasor['abs'].values[-1] - phasor['abs'].values[0]
    assert abs_diff < 20
    phase_diff = phasor['phase'].values[-1] - phasor['phase'].values[0]
    assert phase_diff < 0.001
    print(node + ': ' + str(abs_diff) + '<' + str(phase_diff))

In [ ]:
pt.plot_timeseries(1, ts_dpsim['wr_1'])
pt.plot_timeseries(1, ts_dpsim['wr_2'])
pt.plot_timeseries(1, ts_dpsim['wr_3'])
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (rad/s)')
#plt.ylim([370,380])
#plt.xlim([0,0.06])